In [49]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler,OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score, auc
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import pickle

In [50]:
data = pd.read_csv('train.csv')

In [51]:
data.drop(['day','month'],axis=1,inplace=True)

In [52]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [53]:
X.head()

,age,job,marital,education,default,balance,housing,loan,contact,duration,campaign,pdays,previous,poutcome
0,58,management,married,tertiary,no,2143,yes,no,unknown,261,1,-1,0,unknown
1,44,technician,single,secondary,no,29,yes,no,unknown,151,1,-1,0,unknown
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,76,1,-1,0,unknown
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,92,1,-1,0,unknown
4,33,unknown,single,unknown,no,1,no,no,unknown,198,1,-1,0,unknown


In [54]:
le = LabelEncoder()
X['housing'] = le.fit_transform(X['housing'])
X['loan'] = le.fit_transform(X['loan'])
X['default'] = le.fit_transform(X['default'])

In [55]:
X.head()

,age,job,marital,education,default,balance,housing,loan,contact,duration,campaign,pdays,previous,poutcome
0,58,management,married,tertiary,0,2143,1,0,unknown,261,1,-1,0,unknown
1,44,technician,single,secondary,0,29,1,0,unknown,151,1,-1,0,unknown
2,33,entrepreneur,married,secondary,0,2,1,1,unknown,76,1,-1,0,unknown
3,47,blue-collar,married,unknown,0,1506,1,0,unknown,92,1,-1,0,unknown
4,33,unknown,single,unknown,0,1,0,0,unknown,198,1,-1,0,unknown


In [56]:
cat = ['job','marital','education','contact','poutcome']
one = X[cat]

In [57]:
# categorical_cols = ['job','marital','education','contact','poutcome']
# encoder=OneHotEncoder(sparse=False,handle_unknown='ignore')
# x_categorical=pd.DataFrame(encoder.fit_transform(X[categorical_cols]))
# x_categorical.columns=encoder.get_feature_names_out(categorical_cols)
# # Concatenate encoded categorical features and numerical featur
# x=pd.concat([x_categorical,x.drop(categorical_cols,axis=1)],axis=1)

In [58]:
enc = OneHotEncoder(categories='auto',handle_unknown='ignore')
#enc.fit(np.expand_dims(X['job'].values,axis=-1))
enc.fit(one.values)






OneHotEncoder(handle_unknown='ignore')

In [59]:
enc.categories_

[array(['admin.', 'blue-collar', 'entrepreneur', 'housemaid', 'management',
        'retired', 'self-employed', 'services', 'student', 'technician',
        'unemployed', 'unknown'], dtype=object),
 array(['divorced', 'married', 'single'], dtype=object),
 array(['primary', 'secondary', 'tertiary', 'unknown'], dtype=object),
 array(['cellular', 'telephone', 'unknown'], dtype=object),
 array(['failure', 'other', 'success', 'unknown'], dtype=object)]

In [60]:

en=enc.fit_transform(one.values)

In [61]:
enc.fit(X[cat])

# encode the categorical variables and store the output in a new DataFrame
encoded_array = enc.transform(X[cat]).toarray()
encoded_df = pd.DataFrame(encoded_array, columns=enc.get_feature_names_out(cat))

# concatenate the encoded DataFrame with the original DataFrame
df = pd.concat([X, encoded_df], axis=1)

In [62]:
print(df.columns,df.shape)

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'duration', 'campaign', 'pdays', 'previous',
       'poutcome', 'job_admin.', 'job_blue-collar', 'job_entrepreneur',
       'job_housemaid', 'job_management', 'job_retired', 'job_self-employed',
       'job_services', 'job_student', 'job_technician', 'job_unemployed',
       'job_unknown', 'marital_divorced', 'marital_married', 'marital_single',
       'education_primary', 'education_secondary', 'education_tertiary',
       'education_unknown', 'contact_cellular', 'contact_telephone',
       'contact_unknown', 'poutcome_failure', 'poutcome_other',
       'poutcome_success', 'poutcome_unknown'],
      dtype='object') (43000, 40)


In [63]:
df.drop(['job','marital','education','contact','poutcome'],axis=1,inplace=True)

In [64]:
x_train, x_test, y_train, y_test = train_test_split(df, y, test_size=0.3, random_state=42)

In [65]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x_train, y_train)
Y_pred = model.predict(x_test)
fpr, tpr, _thresholds = metrics.roc_curve(y_test, Y_pred,pos_label=2)
print(classification_report(y_test, Y_pred))


              precision    recall  f1-score   support

           1       0.94      0.90      0.92     11682
           2       0.33      0.48      0.39      1218

    accuracy                           0.86     12900
   macro avg       0.64      0.69      0.65     12900
weighted avg       0.88      0.86      0.87     12900



In [68]:
print(x_test.columns)

Index(['age', 'default', 'balance', 'housing', 'loan', 'duration', 'campaign',
       'pdays', 'previous', 'job_admin.', 'job_blue-collar',
       'job_entrepreneur', 'job_housemaid', 'job_management', 'job_retired',
       'job_self-employed', 'job_services', 'job_student', 'job_technician',
       'job_unemployed', 'job_unknown', 'marital_divorced', 'marital_married',
       'marital_single', 'education_primary', 'education_secondary',
       'education_tertiary', 'education_unknown', 'contact_cellular',
       'contact_telephone', 'contact_unknown', 'poutcome_failure',
       'poutcome_other', 'poutcome_success', 'poutcome_unknown'],
      dtype='object')


In [66]:
with open('model1.pkl','wb') as files:
    pickle.dump(model,files)

In [93]:
arr=['15', '1', '1222', '1', '1', '50', '1', '1', '1', '9', '23', '24', '28', '33', '']
a1=[0]*35
for i in range(8):
    a1[i] = arr[i]
for i in arr[9:-1]:
    a1[int(i)]=1
    
print(a1)


['15', '1', '1222', '1', '1', '50', '1', '1', 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
